# Introduction

this notebook demos example of using llm in a MPS backend (apple silicon GPU) using torch 2.x

Referece:
* torch 2.x MPS Backend: https://pytorch.org/docs/stable/notes/mps.html

In [1]:
import os
import torch
import applyllm as apl

print(apl.__version__)


0.0.6


In [2]:
# check that MPS is availabe (Metal Performance Shaders)
if not torch.backends.mps.is_available():
    print("MPS is not available")
else:
    print("MPS is available")
    mps_device = torch.device("mps")
    print(mps_device)



MPS is available
mps


## Define global variables

In [3]:
from applyllm.accelerators import (
    DirectorySetting,
    TokenHelper as th,
)
    
dir_mode_map = {
    "kf_notebook": DirectorySetting(),
    "mac_local": DirectorySetting(home_dir="/Users/yingding", transformers_cache_home="MODELS", huggingface_token_file="MODELS/.huggingface_token"),
}

model_map = {
    "llama7B-chat":     "meta-llama/Llama-2-7b-chat-hf",
    "llama13B-chat":    "meta-llama/Llama-2-13b-chat-hf",
    "llama70B-chat":    "meta-llama/Llama-2-70b-chat-hf",
    "llama3-8B-inst":   "meta-llama/Meta-Llama-3-8B-Instruct",
    "llama3-70B-inst":  "meta-llama/Meta-Llama-3-70B-Instruct",
    "mistral7B-01":     "mistralai/Mistral-7B-v0.1",
    "mistral7B-inst02": "mistralai/Mistral-7B-Instruct-v0.2",
    "mixtral8x7B-01":   "mistralai/Mixtral-8x7B-v0.1",
    "mixtral8x7B-inst01": "mistralai/Mixtral-8x7B-Instruct-v0.1", 
    "gemma7b-it": "google/gemma-7b-it",
    "gemma7b":    "google/gemma-7b",
    "gemma2b-it": "google/gemma-2b-it",
    "gemma2b":    "google/gemma-2b",
    "gemma7b-it-1.1": "google/gemma-1.1-7b-it",
    "gemma2b-it-1.1": "google/gemma-1.1-2b-it",
    "phi3-medium-128k-inst": "microsoft/Phi-3-medium-128k-instruct",
}

default_model_type = "mistral7B-01"
default_dir_mode = "mac_local"

dir_setting = dir_mode_map[default_dir_mode]

os.environ["WORLD_SIZE"] = "1" 
os.environ['XDG_CACHE_HOME'] = dir_setting.get_cache_home()

print(os.environ['XDG_CACHE_HOME'])

/Users/yingding/MODELS


In [4]:
import transformers
import torch

print(transformers.__version__)
print(torch.__version__)

4.41.0
2.3.0


## Choose LLM model

In [5]:
# model_type = default_model_type
# model_type = "gemma7b-it"
# model_type = "gemma2b-it"
# model_type = "llama3-8B-inst"
model_type = "phi3-medium-128k-inst"
# model_type = "mistral7B-inst02"
# model_type = "llama7B-chat"
# model_type = "llama13B-chat"

model_name = model_map.get(model_type, default_model_type)
print(model_name)

microsoft/Phi-3-medium-128k-instruct


### Fast tokenizer

* https://github.com/huggingface/transformers/issues/23889#issuecomment-1584090357

### Load LLM Model and then Tokenizer

In [6]:
from applyllm.pipelines import (
    ModelCatalog,
    KwargsBuilder
)
token_kwargs = th.gen_token_kwargs(model_type=model_type, dir_setting=dir_setting)

# data_type = torch.bfloat16
if model_name.startswith("microsoft"):
    data_type = "auto"
else:
    data_type = torch.float16

device_map = "mps" # "auto"
# device_map = "auto"

# auto caste not working for mps 4.38.2
# https://github.com/huggingface/transformers/issues/29431 

# mixtral model has no max_new_tokens limit, so it is not set here.
model_kwargs = {
    "torch_dtype": data_type, #bfloat16 is not supported on MPS backend, float16 only on GPU accelerator
    # torch_dtype=torch.float32,
    # max_length=MAX_LENGTH,
    "device_map": device_map,
    "max_length" : None, # remove the total length of the generated response
    "trust_remote_code" : True
}
print(f"model_kwargs: {model_kwargs}")

# set the transformers.pipeline kwargs
# the torch_dtype shall be set both for the model and the pipeline, due to a transformer issue.
# otherwise it will cause unnecessary more memory usage in the pipeline of transformers
# https://github.com/huggingface/transformers/issues/28817
# https://github.com/mlflow/mlflow/pull/10979

# if model_name.startswith("microsoft"):
#     do_sample = False
# else:
#     do_sample = True

do_sample = True

pipeline_kwargs = {
    "task": "text-generation",
    "max_new_tokens" : 200,
    "do_sample" : do_sample, # do_sample True is required for temperature
    "temperature" : 0.01, 
    "device_map" : device_map, # use the MPS device if available
    "top_k": 3,
    "top_p": 0.95,
    # "num_return_sequences": 1,
    "framework": "pt", # use pytorch as framework 
}

gemma_pipeline_kwargs = {
    "add_special_tokens": True,
    "torch_dtype": data_type,
}

# pipeline_kwargs override the model_kwargs during the merge
pipeline_kwargs = KwargsBuilder([model_kwargs]).override(pipeline_kwargs).build()

if model_name.startswith(ModelCatalog.GOOGLE_FAMILY):
    pipeline_kwargs = KwargsBuilder([pipeline_kwargs]).override(gemma_pipeline_kwargs).build()

print(f"pipeline_kwargs: {pipeline_kwargs}")


huggingface token is NOT needed
model_kwargs: {'torch_dtype': 'auto', 'device_map': 'auto', 'max_length': None, 'trust_remote_code': True}
pipeline_kwargs: {'torch_dtype': 'auto', 'device_map': 'auto', 'max_length': None, 'trust_remote_code': True, 'task': 'text-generation', 'max_new_tokens': 200, 'do_sample': True, 'temperature': 0.01, 'top_k': 3, 'top_p': 0.95, 'framework': 'pt'}


In [7]:
torch.mps.driver_allocated_memory()

393216

In [8]:
driver_allocated_mem = f"{int(torch.mps.driver_allocated_memory()/1024**3)}GB"
lm_allocated_mem = f"{int(torch.mps.current_allocated_memory()/1024**3)}GB"
print(driver_allocated_mem)
print(lm_allocated_mem)

0GB
0GB


### Max memory to offload parts of LLM model to the CPU memory
* https://huggingface.co/docs/accelerate/concept_guides/big_model_inference#designing-a-device-map

Note:
* Max Memory offload to CPU is CUDA implementation only



In [9]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from applyllm.utils import time_func
from applyllm.pipelines import ModelConfig, LocalCausalLMConfig

# For a M1 max with 64GB memory, set the limit to 48GB
# cuda_max_memory = {
#   0: "48GB", # GPU device 0
#   "cpu": "1GB", # CPU device with no memory, since M1 max has unified memory
# } 

base_lm_config = ModelConfig(
  model_config = {
    "pretrained_model_name_or_path": model_name,
    "device_map": device_map,
    "trust_remote_code" : True
    # "max_memory": cuda_max_memory,
  }
)

# No bitsandbytes qunatization support for MPS backend yet, set quantized to False
kwargs = {
  "quantized": False,
  "model_config": base_lm_config.get_config(),
  "quantization_config": {
    "quantization_config": transformers.BitsAndBytesConfig(
      load_in_4bit=True,
      bnb_4bit_quant_type='nf4',
      bnb_4bit_use_double_quant=True,
      bnb_4bit_compute_dtype=torch.bfloat16
      )
    }
}

lm_config = LocalCausalLMConfig(**kwargs)

@time_func
def load_model():
  return AutoModelForCausalLM.from_pretrained(    
    **lm_config.get_config(),
    **token_kwargs,  
  )

model = load_model()

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

executed: load_model() python function
walltime: 29.37050700187683 in secs.


In [10]:
tokenizer_kwargs = {
    "model_config": {
        "pretrained_model_name_or_path": model_name,
        "device": "cpu",
        # "device_map": "auto", # put to GPU if GPU is available
        # "max_position_embeddings": MAX_LENGTH,
        # "max_length": MAX_LENGTH,
    },
}
tokenizer_config = ModelConfig(**tokenizer_kwargs)

tokenizer = AutoTokenizer.from_pretrained(
    **tokenizer_config.get_config(),
    **token_kwargs
)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [11]:
tokenizer

LlamaTokenizerFast(name_or_path='microsoft/Phi-3-medium-128k-instruct', vocab_size=32000, model_max_length=131072, is_fast=True, padding_side='left', truncation_side='right', special_tokens={'bos_token': '<s>', 'eos_token': '<|endoftext|>', 'unk_token': '<unk>', 'pad_token': '<|endoftext|>'}, clean_up_tokenization_spaces=False),  added_tokens_decoder={
	0: AddedToken("<unk>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("<s>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("</s>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=False),
	32000: AddedToken("<|endoftext|>", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	32001: AddedToken("<|assistant|>", rstrip=True, lstrip=False, single_word=False, normalized=False, special=True),
	32002: AddedToken("<|placeholder1|>", rstrip=True, lstrip=False, single_word=False, normalized=False, sp

In [12]:
print(type(tokenizer))

<class 'transformers.models.llama.tokenization_llama_fast.LlamaTokenizerFast'>


### Testing token
* https://huggingface.co/docs/tokenizers/pipeline

## Phi instruct template
* https://huggingface.co/microsoft/Phi-3-small-128k-instruct

In [13]:
print(model_name)

microsoft/Phi-3-medium-128k-instruct


In [14]:
from langchain import PromptTemplate
from applyllm.pipelines import (
    ModelCatalog,
    PromptHelper
)

model_info = ModelCatalog.get_model_info(model_name)
prompt_helper = PromptHelper(model_info)



if model_info.model_family == ModelCatalog.GOOGLE_FAMILY:
    query = """BEGIN EXAMPLE
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
END EXAMPLE

Your turn:            
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have? 
"""
    inputs=[prompt_helper.gen_prompt(query)]
elif model_name.startswith("microsoft"):
    # msft_template = "<|endoftext|><|user|>\n{query}<|end|>\n<|assistant|>"
    msft_template = "<|user|>\n{query}<|end|>\n<|assistant|>"
    user_query = """
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""
    template = PromptTemplate(template=msft_template, input_variables=["query"])
    inputs=[template.format(query=user_query)]
else: 
    inputs=["""
Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
"""]

In [15]:
input_test_encoded = tokenizer.encode(inputs[0])
print(f"{len(input_test_encoded)}")
print(input_test_encoded)

124
[32010, 660, 29901, 14159, 756, 29871, 29941, 22556, 26563, 29889, 940, 1321, 952, 29871, 29906, 901, 508, 29879, 310, 22556, 26563, 29889, 7806, 508, 756, 29871, 29946, 22556, 26563, 29889, 1128, 1784, 22556, 26563, 947, 540, 505, 1286, 29973, 13, 29909, 29901, 14159, 4687, 411, 29871, 29941, 26563, 29889, 29871, 29906, 508, 29879, 310, 29871, 29946, 22556, 26563, 1269, 338, 29871, 29947, 22556, 26563, 29889, 29871, 29941, 718, 29871, 29947, 353, 29871, 29896, 29896, 29889, 450, 1234, 338, 29871, 29896, 29896, 29889, 13, 29984, 29901, 450, 274, 2142, 1308, 423, 750, 29871, 29906, 29941, 623, 793, 29889, 960, 896, 1304, 29871, 29906, 29900, 304, 1207, 301, 3322, 322, 18093, 29871, 29953, 901, 29892, 920, 1784, 623, 793, 437, 896, 505, 29973, 13, 32007, 32001]


In [16]:
response_test_decoded = tokenizer.decode(input_test_encoded)
print(response_test_decoded)

<|user|> Q: Roger has 3 tennis balls. He buys 2 more cans of tennis balls. Each can has 4 tennis balls. How many tennis balls does he have now?
A: Roger started with 3 balls. 2 cans of 4 tennis balls each is 8 tennis balls. 3 + 8 = 11. The answer is 11.
Q: The cafeteria had 23 apples. If they used 20 to make lunch and bought 6 more, how many apples do they have?
<|end|><|assistant|>


### Load LLM

In [17]:
# bitsandbytes quantization does not work with MPS backend
print(pipeline_kwargs)

# transformer pipeline kwargs
tp_kwargs = {
    "model": model,
    "tokenizer": tokenizer,
}

tp_config = ModelConfig(model_config = tp_kwargs)

generator = transformers.pipeline(
    **tp_config.get_config(),
    **pipeline_kwargs,
    **token_kwargs,
    # **compression_kwargs,
)

{'torch_dtype': 'auto', 'device_map': 'auto', 'max_length': None, 'trust_remote_code': True, 'task': 'text-generation', 'max_new_tokens': 200, 'do_sample': True, 'temperature': 0.01, 'top_k': 3, 'top_p': 0.95, 'framework': 'pt'}


##### Install autopep8 or black extension in VSCode
`shift + opt + F` to auto format python code

In [18]:
from applyllm.accelerators import AcceleratorStatus

gpu_status = AcceleratorStatus.create_accelerator_status()
gpu_status.gpu_usage()

--------------------
Allocated memory : 53.965210 GB
--------------------


In [19]:
import pydantic, time
pydantic.__version__

'1.10.13'

In [20]:
from pprint import pprint

def chat_gen(
    generator: transformers.pipelines.text_generation.TextGenerationPipeline, 
    tokenizer,
    gpu_status: AcceleratorStatus
):    
    def local(input_prompts: list=[], temperature: float=0.01, max_new_tokens: int=200, verbose: bool=True) -> list:
        """
        do_sample, top_k, num_return_sequences, eos_token_id are the settings 
        the TextGenerationPipeline
        
        Reference:
        https://huggingface.co/docs/transformers/generation_strategies#customize-text-generation
        """
        start = time.time()
        model_dependeny_kwargs = {}
        if model_name.startswith(ModelCatalog.GOOGLE_FAMILY):
            # for gemma 
            model_dependeny_kwargs = {
                "add_special_tokens": True,
            }
        if model_name.startswith(ModelCatalog.MISTRAL_FAMILY):
            model_dependeny_kwargs = {
                "pad_token_id": tokenizer.eos_token_id,
            }
        if model_name.startswith(ModelCatalog.META_FAMILY):
            model_dependeny_kwargs = {
                "eos_token_id" : tokenizer.eos_token_id
            }
        sequences = generator(
            input_prompts,
            do_sample=do_sample,
            top_k=3,
            top_p=0.95,
            # num_return_sequences=1,
            # pad_token_id=tokenizer.eos_token_id, # for mistral
            # eos_token_id=tokenizer.eos_token_id, # for llama
            # max_length=200,
            max_new_tokens= max_new_tokens, # 200 # max number of tokens to generate in the output
            temperature=temperature,
            repetition_penalty=1.15,  # without this output begins repeating
            **model_dependeny_kwargs,
        )
        # for seq in sequences:
        #     print(f"Result: \n{seq['generated_text']}")
        
        batch_result = []
        for prompt_result in sequences: # passed a list of prompt
            result = []
            for seq in prompt_result: # 
                result.append(f"Result: \n{seq['generated_text']}")
            batch_result.append(result)
            
        end = time.time()
        duration = end - start
        
        if verbose == True:
            for prompt_result in batch_result:
                for result in prompt_result:
                    print("promt-response")
                    # pprint(result)
                    print(result)
            print("-"*20)
            print(f"walltime: {duration} in secs.")
            gpu_status.gpu_usage()
            
        return batch_result   
    return local
    
chat = chat_gen(generator, tokenizer, gpu_status)

In [21]:
verbose = True
batch_answers = chat(inputs, temperature=0.01, max_new_tokens = 80, verbose=verbose)

if not verbose:
    prompt_0_results = batch_answers[0]
    print(prompt_0_results[0])

You are not running the flash-attention implementation, expect numerical differences.


KeyboardInterrupt: 

### mlflow autologging langchain
* https://mlflow.org/docs/latest/llms/langchain/guide/index.html+
* https://github.com/mlflow/mlflow/issues/9237#issuecomment-1667549626

#### Issue
* HuggingFacePipeline is not callable from mlflow run: https://github.com/langchain-ai/langchain/issues/8858

#### LangChain Callback Handler
* https://python.langchain.com/docs/integrations/providers/aim_tracking
* https://python.langchain.com/docs/integrations/providers/mlflow_tracking
* https://python.langchain.com/docs/integrations/providers/mlflow_ai_gateway
* https://python.langchain.com/docs/integrations/providers/mlflow
* https://api.python.langchain.com/en/latest/_modules/langchain_community/callbacks/mlflow_callback.html

In [ ]:
import os
# os.environ["MLFLOW_TRACKING_URI"] = "./mlruns"

In [ ]:
import mlflow
import logging
import time
from pprint import pprint

logging.getLogger("mlflow").setLevel(logging.DEBUG)

from langchain import PromptTemplate, LLMChain, HuggingFaceHub
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline
# from langchain.callbacks import MlflowCallbackHandler

# Set the run name to time string
run_name = time.strftime("%Y-%m-%d_%H-%M-%S")
experiment_name = "langchain"
search_pattern = f"name = '{experiment_name}'"
experiments = mlflow.search_experiments(filter_string=search_pattern)

if len(experiments) < 1:
    experiment_id = mlflow.create_experiment(name=experiment_name)
    print(f"experiment with string id {experiment_id} is created.")
else:
    experiment_id = experiments[0].experiment_id
    # experiment_id = experiments.experiment_id[0]
    print(f"experiment with string id {experiment_id} is reused.")

# mlflow_callback = MlflowCallbackHandler(experiment=experiment_name, name=run_name)

mlflow.end_run()
mlflow.set_experiment(experiment_id=experiment_id)
mlflow.start_run(run_name=run_name)


llm = HuggingFacePipeline(
    pipeline=generator 
)

template = prompt_helper.gen_prompt("{input}")
prompt = PromptTemplate(template=template, input_variables=["input"])

mlflow.log_param("system_prompt", template)

llm_chain = LLMChain(prompt=prompt, llm=llm)
# llm_chain = LLMChain(prompt=prompt, llm=llm, callbacks=[mlflow_callback])

# mlflow_callback.flush_tracker(llm_chain)

# print(llm_chain.invoke({"input": inputs[0]}))
# format the output of print with multiple lines of 60 max line length
# response = llm_chain.run(inputs[0])
dict_response = llm_chain.invoke(input={"input":inputs[0]})
response = dict_response.get("text", "")

print(repr(llm_chain))

mlflow.log_param("response", response)

# Evaluate the model on some example questions
import pandas as pd
eval_data = pd.DataFrame(
    {
        "input": [
            "What is MLflow?",
            "What is Spark?",
        ],
        "ground_truth": [
            "MLflow is an open-source platform for managing the end-to-end machine learning (ML) " +
            "lifecycle. It was developed by Databricks, a company that specializes in big data and " +
            "machine learning solutions. MLflow is designed to address the challenges that data " +
            "scientists and machine learning engineers face when developing, training, and deploying " +
            "machine learning models.",
            "Apache Spark is an open-source, distributed computing system designed for big data " +
            "processing and analytics. It was developed in response to limitations of the Hadoop " +
            "MapReduce computing model, offering improvements in speed and ease of use. Spark " +
            "provides libraries for various tasks such as data ingestion, processing, and analysis " +
            "through its components like Spark SQL for structured data, Spark Streaming for " +
            "real-time data processing, and MLlib for machine learning tasks",
        ],
    }
)

print(eval_data)

class LocalHfpModel():
    """local huggingface pipeline model"""
    def __init__(self, llm_chain):
        self.llm_chain = llm_chain
    

    def __call__(self, data):
        # single call returns string
        # response = self.llm_chain.run(data["input"].tolist())
        # self.results.append(response)
        # GPU batch
        response = self.llm_chain.batch(data["input"].tolist())
        # print(type(response))
        # print(response)
        return [ _dict["text"] for _dict in response]

# load the LocalHfpModel() to mlflow.evaluate
results = mlflow.evaluate(
    model=LocalHfpModel(llm_chain),
    model_type="question-answering",
    targets="ground_truth",
    data=eval_data,
)
print(f"See aggregated evaluation results below: \n{results.metrics}")

# Evaluation result for each data record is available in `results.tables`.
eval_table = results.tables["eval_results_table"]
print(f"See evaluation table below: \n{eval_table}")

mlflow.end_run()

pprint(response, indent=0, width=100)

In [ ]:
# # Set the run name to time string
# run_name = time.strftime("%Y-%m-%d_%H-%M-%S")
# experiment_name = "local_llm_test"
# search_pattern = f"name = '{experiment_name}'"
# experiments = mlflow.search_experiments(filter_string=search_pattern)

# if len(experiments) < 1:
#     experiment_id = mlflow.create_experiment(name=experiment_name)
#     print(f"experiment with string id {experiment_id} is created.")
# else:
#     experiment_id = experiments[0].experiment_id
#     # experiment_id = experiments.experiment_id[0]
#     print(f"experiment with string id {experiment_id} is reused.")

    
# try:
#     with mlflow.start_run(experiment_id=experiment_id, run_name=run_name) as run:
#         logged_model = mlflow.langchain.log_model(
#             lc_model=llm_chain,
#             artifact_path="models")
        
#     # Load the logged model using MLflow's Python function flavor
#     loaded_model = mlflow.pyfunc.load_model(logged_model.model_uri)

#     # Predict using the loaded model, with defined input schema from prompt template
#     print(loaded_model.predict([{"input": inputs[0]}]))
# except Exception as e:
#     print(e)
#     mlflow.end_run()


In [ ]:
# We automatically log the model and trace related artifacts
# A model with name `lc_model` is registered, we can load it back as a PyFunc model
# model_name = "lc_model"
# model_version = 1
# loaded_model = mlflow.pyfunc.load_model(f"models:/{model_name}/{model_version}")
# print(loaded_model.predict(inputs))

In [ ]:
import gc
def clear_mps_memory(tokenizer, generator):
    """clear the MPS memory"""
    if tokenizer is not None:
        del tokenizer
    if generator is not None:
        # need to move the model to cpu before delete.
        generator.model.cpu()
        del generator
    gc.collect()
    torch.mps.empty_cache()
    # report the GPU usage
    gpu_status.gpu_usage()


In [ ]:
CLEAR_MEMORY = False
# CLEAR_MEMORY = True

if CLEAR_MEMORY:
    clear_mps_memory(tokenizer=tokenizer, generator=generator)

In [ ]:
gpu_status.gpu_usage()

In [ ]:
# inputs2 = ["Which animal is the largest mammal?"]
inputs2 = ["Can you tell me something about chron's disease?"]

# hallucination https://www.findacode.com/snomed/34000006--crohns-disease.html
'''
promt-response
Result: 
Which snomed ct code has chron's disease?

The SNOMED CT Code for Chronic Disease is 4621830.
--------------------
walltime: 4.950197219848633 in secs.
--------------------
Allocated memory : 54.158737 GB
--------------------
'''
# real answer is 34000006, probably need a RAG 
# inputs2 = ["Which snomed ct code has chron's disease?"]

# inputs2 = ["Can you tell me more about the company nordcloud?"]
# inputs2 = ["Can you tell me more about the company nordcloud in munich?"]

In [ ]:
responses = chat(inputs2, temperature=0.01, max_new_tokens = 80, verbose=verbose)